# Lab 3 - Spatial Prediction Pipeline with ArcPy
## Elevation
### Luke Zaruba
### GIS 5572: ArcGIS II
### 2023-04-06

In this notebook, an automated end-to-end pipeline is being implemented which identifies the best interpolation method based on accuracy, calculates point accuracy at locations with known values, aggregates the values to H3 hexagons (making it easier to work with in Postgres in GCP), before exporting the layers to the Postgres database on GCP.

In [1]:
# Import Packages
import sys

# Import Pipeline class to run pipeline
# Need to Append Path for Pro - Can use '..' outside of Pro in IDE/Editor (if using structure of repo)
sys.path.append(r"C:\gitFiles\GIS5572\Lab3")
from utils.interpolation import Pipeline

In [2]:
# Define Variables
elevation_points = r"C:\Users\lukea\Documents\ArcGIS\Projects\lab2_gis5572\lab2_gis5572.gdb\elevation1km_pt"
output_directory = r"C:\Users\lukea\Documents\ArcGIS\Projects\GIS5572 - Lab 3"
output_gdb = r"C:\Users\lukea\Documents\ArcGIS\Projects\GIS5572 - Lab 3\GIS5572 - Lab 3.gdb"
sde_connection = r"C:\Users\lukea\Documents\ArcGIS\Projects\GIS5572 - Lab 3\lab3.sde"

## Run Pipeline

In [3]:
# Create Elevation Pipeline Object
elevationPipeline = Pipeline(elevation_points, output_directory, output_gdb, "grid_code")

In [4]:
# Run Exploratory Interpolation
elevationPipeline.run_exploratory_interpolation()

Start Time: Thursday, March 30, 2023 9:36:25 PM
Calculating Ordinary Kriging – Default
Calculating Ordinary Kriging – Optimized
Calculating Universal Kriging – Default
Calculating Universal Kriging – Optimized
Calculating Inverse Distance Weighted - Default
Calculating Inverse Distance Weighted - Optimized
 

--------------------------------------------
RANK | NAME
--------------------------------------------


1    | Universal Kriging – Optimized


2    | Universal Kriging – Default


3    | Inverse Distance Weighted - Optimized


4    | Ordinary Kriging – Default


5    | Ordinary Kriging – Optimized


6    | Inverse Distance Weighted - Default
--------------------------------------------
Succeeded at Thursday, March 30, 2023 9:37:43 PM (Elapsed Time: 1 minutes 17 seconds)


In [5]:
# Display Accuracy Assessment Table
elevationPipeline.display("DATAFRAME")

,OID_,DESCR,RANK,INCLUDED,RMSE,ME,ME_STD,RMSE_STD,ASE,MAX_ERROR,PERC_ERROR
0,1,Universal Kriging – Optimized,1,Yes,25.641630,-0.017781,-0.001510,1.548306,16.519436,426.938871,88.173274
1,2,Universal Kriging – Default,2,Yes,25.821678,-0.119737,-0.004904,1.016500,25.367149,413.368171,88.090230
2,3,Inverse Distance Weighted - Optimized,3,Yes,26.130605,0.109924,NaN,NaN,NaN,457.500000,87.947744
3,4,Ordinary Kriging – Default,4,Yes,26.145563,-0.010077,-0.000726,0.915134,28.518310,406.549347,87.940844
4,5,Ordinary Kriging – Optimized,5,Yes,26.616510,0.044801,0.001157,1.266233,20.931003,442.806438,87.723629
5,6,Inverse Distance Weighted - Default,6,Yes,27.079272,0.162070,NaN,NaN,NaN,403.914474,87.510188


In [6]:
# Create Point Accuracy Layer
elevationPipeline.create_point_accuracy_layer()

Point accuracy successfully generated at: C:\Users\lukea\Documents\ArcGIS\Projects\GIS5572 - Lab 3\GIS5572 - Lab 3.gdb\elevation1km_pt_point_diff


In [7]:
# Convert Interpolation Results to H3
elevationPipeline.convert_results_to_hex()

Data successfully aggregated to H3 hexagons at: C:\Users\lukea\Documents\ArcGIS\Projects\GIS5572 - Lab 3\GIS5572 - Lab 3.gdb\elevation1km_pt_h3_6


In [8]:
# Export H3 Hexagons to SDE
elevationPipeline.export_to_sde(sde_connection, "TESSELLATION")

In [9]:
# Export Point Accuracy Points to SDE
elevationPipeline.export_to_sde(sde_connection, "POINT_ACCURACY")